In [10]:
import numpy as np
from collections import Counter
import math
import sys
import random


In [11]:
# inputfile = "sudoku/s10b.txt"
iteration_number = 10 
r = 0.05
ants = 10

In [12]:
def ACO_Algorithm(a, t, ants):
    sudoku_problem = a.copy()   
    bestfitness = 0             #initialize the best fitness
    bestsolution = []           # initialize the best solution
    for i in range (1, ants+1):             # create all ants
        fitness, solution = ant(sudoku_problem,t)  # retrieve the solution and fitness from the ant given the problem and pheramone matrix
        if fitness > bestfitness:           # compare fitness to best fitness so far 
            bestfitness = fitness           # save new fitness value if greater than previous
            bestsolution = solution.copy()  # copy the solutions
            if np.count_nonzero(solution) == 81:   # if there is no zero left then the answer is found 
                break                       
    return bestfitness, bestsolution        # return best solution and fitness for one iteration


In [13]:
def update_solution(ant_solution, digits, row_matrix, digit_submatrix, column_matrix):
    # for each position if there are obvious digits to fill, Fill the digit with that number
    for (i,j), p in np.ndenumerate(digits):
        # calculate index z for digit_submatrix
        if i < 3: 
            z = int((j/3)+1)
        elif 2 < i < 6: 
            z = int((j/3)+1)+3
        elif 5 < i: 
            z = int((j/3)+1)+6

        # calculate the list of digits that are taken for this position
        digit_list = list(row_matrix[i] + column_matrix[j] + digit_submatrix[z-1])

        # count how many are available to fill
        digit_possible = digit_list.count(0) 

        # if there are possible digits, save the number in the digits matrix
        if ant_solution[i][j] > 0: 
            digit_possible = 0
        digits[i][j]=digit_possible

        # if only one digit is possible, find it and put it in the solution
        if digit_possible == 1 and ant_solution[i][j] == 0:
            ant_solution[i][j]=digit_list.index(0)+1
            digits[i][j]=0
            break

    return ant_solution, digits

In [14]:
def update_memory(ant_solution, row_matrix, digit_submatrix, column_matrix):
    # Create a memory of taken numbers in submatrixes, rows and columns
    count = 0
    for x in [0,3,6]:
        for y in [0,3,6]:
            for (i2,j2),sd in np.ndenumerate(ant_solution[x:x+3,y:y+3]):
                if sd > 0:
                    digit_submatrix[count][sd-1]=1
            count += 1
    # for each row and column, check which numbers are already taken
    for (i,j), digit in np.ndenumerate(ant_solution):
        if digit > 0:
            row_matrix[i][digit-1] = 1
            column_matrix[j][digit-1] = 1

    return row_matrix, digit_submatrix, column_matrix

In [15]:

def ant(d,t):
    #initialize the ants solution set 
    ant_solution = d.copy()   

    current_fitness = 0
    row_matrix = np.zeros((9,9)) # initialize digit matrix in each row
    column_matrix = np.zeros((9,9)) # initialize digit matrix in each column
    digit_submatrix = np.zeros((9,9), dtype=np.int) # initialize the matrix for possible digits in each 3x3 submatrix
    digits = np.zeros((9,9)) # initialize the possible digits matrix
    while (True):
        
        row_matrix, digit_submatrix, column_matrix = update_memory(ant_solution, row_matrix, digit_submatrix, column_matrix)
        ant_solution, digits = update_solution(ant_solution, digits, row_matrix, digit_submatrix, column_matrix)
        
        # if there are no zeros in ant_solution answer is found, kill the ant and return the solution
        if np.count_nonzero(ant_solution) == 81: 
            return current_fitness, ant_solution
        
        #if no more steps are possible based on analytics
        if(np.count_nonzero(ant_solution)==current_fitness):
            w = np.zeros((9,9,9)) # initialize weight matrix
            wsum = 0 # sum of weights
            # for each weight, fill in the actual weight
            for (i,j,k), v in np.ndenumerate(w):
                if i < 3: 
                    z = int((j/3)+1)
                elif 2 < i < 6: 
                    z = int((j/3)+1)+3
                elif 5 < i: 
                    z = int((j/3)+1)+6
                
                if digits[i][j] > 0:
                    # weight equation: pheromone * 10-number of possible places for the digit * 10-number of possible digits for the position
                    w[i][j][k] = t[i][j][k]*(10-list(digit_submatrix[z-1]).count(0))*(10-digits[i][j])
                    # sum up the weights
                    wsum += w[i][j][k]
            
            # in case there are no more options left return
            if wsum == 0: 
                return current_fitness, ant_solution
            
            # calculate a random value between 0 and the sum of weights and normalize it
            next_selection = random.randint(0,int(wsum))/wsum
            # iterate through possible digits until the random value has been reached, effectively a weighted roulette wheel
            psum = 0
            for (i,j,k), v in np.ndenumerate(w):
                psum += w[i][j][k]/wsum
                # if this digit is the chosen one.. 
                if psum > next_selection:
                    # generate z index for 3x3
                    if i < 3: z = int((j/3)+1)
                    if 2 < i < 6: z = int((j/3)+1)+3
                    if 5 < i: z = int((j/3)+1)+6
                    # check for validity and add to solution
                    if row_matrix[i][k] == 0 and column_matrix[j][k] == 0 and digit_submatrix[z-1][k] == 0:
                        ant_solution[i][j] = k+1
                    else:
                        return current_fitness, ant_solution
                    
                    break
        else:
            current_fitness = np.count_nonzero(ant_solution)          
    return ant_solution, current_fitness

In [17]:
if __name__ == '__main__':
    #open inputfile and take sudoku into problem_matrix
    inputfiles = ["sudoku/s10a.txt", "sudoku/s10b.txt", "sudoku/s11a.txt","sudoku/s11b.txt"]
    for inputfile in inputfiles:
        print(inputfile)
        # Create the matrixes;
        problem_matrix = np.zeros((9,9), dtype=np.int)  # initialize problem matrix    
        global_pheromone= np.ones((9,9,9))*1000           # initialize pheromene matrix
        globalbestfitness = 0                           # initialize global best fitness
        globalbestsolution = []                         # initialize global best solution matrix

        with open(inputfile, "r") as f:
            for i,line in enumerate(f):
                digitlist = line.split()
                if not digitlist == []:
                    for j,digitstr in enumerate(digitlist):
                        problem_matrix[i][j] = int(digitstr)


        for iteration in range (1,iteration_number):
            # the best solution & fitness for iteration
            bestfitnessofiteration, bestsolutionofiteration = ACO_Algorithm(problem_matrix,global_pheromone,ants)
            # if it is better than the global best, copy it
            if(bestfitnessofiteration>globalbestfitness):
                globalbestfitness = bestfitnessofiteration
                globalbestsolution = bestsolutionofiteration.copy()
            # if it is solved, stop iteration
            if(np.count_nonzero(globalbestsolution) == 81):
                print ('success')
                break
            # equation for new pheromone to be added
            add_pheromone = (bestfitnessofiteration/81)*(bestfitnessofiteration/81)
            # pheromone decay according to rate r
            for (i,j,k), phe in np.ndenumerate(global_pheromone):
                global_pheromone[i][j][k] = (1-r)*global_pheromone[i][j][k]
            # add new pheromone for best solution set
            for (i,j), k in np.ndenumerate(bestsolutionofiteration):
                if int(k) > 0:
                    # pheromone update equation, the constant can be modified to change behavior
                    global_pheromone[i][j][int(k-1)] += 1000*add_pheromone

        print(globalbestsolution)
        print('Number of iterations:' + str(iteration))

sudoku/s10a.txt
success
[[9 7 2 8 6 3 5 4 1]
 [6 1 8 7 4 5 9 2 3]
 [4 5 3 2 9 1 6 8 7]
 [5 4 9 1 2 8 7 3 6]
 [8 2 1 6 3 7 4 5 9]
 [7 3 6 4 5 9 2 1 8]
 [2 9 5 3 8 6 1 7 4]
 [1 8 4 9 7 2 3 6 5]
 [3 6 7 5 1 4 8 9 2]]
Number of iterations:1
sudoku/s10b.txt
success
[[8 5 2 7 1 6 9 4 3]
 [1 9 7 8 4 3 6 5 2]
 [4 6 3 9 2 5 1 8 7]
 [2 7 8 6 3 4 5 9 1]
 [6 4 5 1 7 9 3 2 8]
 [9 3 1 5 8 2 4 7 6]
 [7 8 6 4 9 1 2 3 5]
 [3 1 4 2 5 8 7 6 9]
 [5 2 9 3 6 7 8 1 4]]
Number of iterations:1
sudoku/s11a.txt
success
[[3 4 5 8 7 1 2 6 9]
 [2 7 9 6 5 3 1 8 4]
 [8 6 1 4 2 9 5 3 7]
 [1 9 7 3 4 6 8 5 2]
 [4 5 2 7 1 8 3 9 6]
 [6 8 3 5 9 2 7 4 1]
 [7 3 8 2 6 4 9 1 5]
 [5 1 6 9 3 7 4 2 8]
 [9 2 4 1 8 5 6 7 3]]
Number of iterations:2
sudoku/s11b.txt
[[2 3 1 4 6 9 7 8 5]
 [7 5 8 2 1 3 6 4 9]
 [9 6 4 7 8 5 1 2 3]
 [5 4 3 8 9 7 2 1 6]
 [1 7 6 5 2 0 8 9 4]
 [8 9 2 1 4 6 5 3 7]
 [6 1 9 3 7 8 4 5 2]
 [3 2 7 6 5 4 9 0 8]
 [4 8 5 9 0 2 3 6 1]]
Number of iterations:9
